<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/branch1/Nettoyages_des_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On peut Définir la seed pour que l'aléatoire soit consistant entre exécution

In [1]:
seed=42

Les pip install

Il faut lancer ce bloc de code 2 fois

In [2]:
%pip install --upgrade pandas

In [3]:
pip install emot --upgrade

     |████████████████████████████████| 61 kB 17 kB/s 


In [4]:
import pickle
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/PFE/"
with open(DATA_PATH+"data_tweets.txt", "rb") as file :  
    data_tweets = pickle.load(file)

Mounted at /content/drive


In [5]:
import emot
import re
import numpy as np

In [6]:
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6 MB 21.6 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-py3-none-any.whl size=90338488 sha256=53fdd9298df7ad9cfcfc56bdd7943ba50cff8887ec035a7ac260fc3d6bd827e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-s853c9m7/wheels/2e/26/ff/ce93eb966e7176ebe81e6c98209582e13e108cdd2d6d636df0
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [7]:
import spacy
import fr_core_news_md

In [8]:
nlp = fr_core_news_md.load()

On sélectionne aléatoirement 10000 pour chaque catégories restantes

In [9]:
"""# Création des groupes à comparer
notfiables = [2] #Groupe des non fiables
df_notfiables = data_tweets[data_tweets['liability'].isin(notfiables)]
maybefiables = [3] #Groupe peut-être fiable
df_maybefiables = data_tweets[data_tweets['liability'].isin(maybefiables)]
isfiables = [4] #Groupe des fiables
df_isfiables = data_tweets[data_tweets['liability'].isin(isfiables)]"""

"# Création des groupes à comparer\nnotfiables = [2] #Groupe des non fiables\ndf_notfiables = data_tweets[data_tweets['liability'].isin(notfiables)]\nmaybefiables = [3] #Groupe peut-être fiable\ndf_maybefiables = data_tweets[data_tweets['liability'].isin(maybefiables)]\nisfiables = [4] #Groupe des fiables\ndf_isfiables = data_tweets[data_tweets['liability'].isin(isfiables)]"

In [10]:
"""np.random.default_rng(seed)
#Faire un random seed dans random.choice pour conservé

#10000 tweets non fiables sélectionnés aléatoirement
remove_n = df_notfiables.shape[0]-10000
drop_indices = np.random.choice(df_notfiables.index, remove_n, replace=False)
df_notfiables = df_notfiables.drop(drop_indices)
#10000 tweets peut-être fiables sélectionnés aléatoirement
remove_n = df_maybefiables.shape[0]-10000
drop_indices = np.random.choice(df_maybefiables.index, remove_n, replace=False)
df_maybefiables = df_maybefiables.drop(drop_indices)
#10000 tweets fiables sélectionnés aléatoirement
remove_n = df_isfiables.shape[0]-10000
drop_indices = np.random.choice(df_isfiables.index, remove_n, replace=False)
df_isfiables = df_isfiables.drop(drop_indices)"""

'np.random.default_rng(seed)\n#Faire un random seed dans random.choice pour conservé\n\n#10000 tweets non fiables sélectionnés aléatoirement\nremove_n = df_notfiables.shape[0]-10000\ndrop_indices = np.random.choice(df_notfiables.index, remove_n, replace=False)\ndf_notfiables = df_notfiables.drop(drop_indices)\n#10000 tweets peut-être fiables sélectionnés aléatoirement\nremove_n = df_maybefiables.shape[0]-10000\ndrop_indices = np.random.choice(df_maybefiables.index, remove_n, replace=False)\ndf_maybefiables = df_maybefiables.drop(drop_indices)\n#10000 tweets fiables sélectionnés aléatoirement\nremove_n = df_isfiables.shape[0]-10000\ndrop_indices = np.random.choice(df_isfiables.index, remove_n, replace=False)\ndf_isfiables = df_isfiables.drop(drop_indices)'

In [11]:
"""df_notfiables['liability'] = 1
df_maybefiables['liability'] = df_notfiables['liability'].iloc[0]
df_isfiables['liability'] = 0
df_notfiables['liability_label'] = 'site plutôt douteux'
df_maybefiables['liability_label'] = df_notfiables['liability_label'].iloc[0]
df_isfiables['liability_label'] = 'site plutôt fiable'"""

"df_notfiables['liability'] = 1\ndf_maybefiables['liability'] = df_notfiables['liability'].iloc[0]\ndf_isfiables['liability'] = 0\ndf_notfiables['liability_label'] = 'site plutôt douteux'\ndf_maybefiables['liability_label'] = df_notfiables['liability_label'].iloc[0]\ndf_isfiables['liability_label'] = 'site plutôt fiable'"

In [12]:
"""# df_group = pd.concat([df_notfiables, df_isfiables]) #fausse informations vs fiable
# df_group = pd.concat([df_maybefiables, df_isfiables]) #fiabilité douteuse vs fiable
# df_group = pd.concat([df_notfiables, df_maybefiables]) #fausse informations vs fiabilité douteuse; Attention, dans la suite du programme les noms peuvent sembler bizarre pour cette ligne
df_group = pd.concat([df_notfiables.head(5000), df_maybefiables.head(5000), df_isfiables]) #faux+doute vs fiable

df_group = df_group.drop(columns=['user_id',
                                  'id_tweet',
                                  'created_at',
                                  'retweet_count',
                                  'favorite_count',
                                  'count_followers'
                                  ]
                         ).reset_index(drop=True)

df_group = df_group[['liability', 
                     'liability_label', 
                     'tweet', 
                     'description',
                     'user_screen_name'
                     ]]"""

"# df_group = pd.concat([df_notfiables, df_isfiables]) #fausse informations vs fiable\n# df_group = pd.concat([df_maybefiables, df_isfiables]) #fiabilité douteuse vs fiable\n# df_group = pd.concat([df_notfiables, df_maybefiables]) #fausse informations vs fiabilité douteuse; Attention, dans la suite du programme les noms peuvent sembler bizarre pour cette ligne\ndf_group = pd.concat([df_notfiables.head(5000), df_maybefiables.head(5000), df_isfiables]) #faux+doute vs fiable\n\ndf_group = df_group.drop(columns=['user_id',\n                                  'id_tweet',\n                                  'created_at',\n                                  'retweet_count',\n                                  'favorite_count',\n                                  'count_followers'\n                                  ]\n                         ).reset_index(drop=True)\n\ndf_group = df_group[['liability', \n                     'liability_label', \n                     'tweet', \n                    

In [13]:
df_group = data_tweets.copy()
df_group = df_group.drop(columns=['user_id',
                                  'id_tweet',
                                  'created_at'
                                  ]
                         ).reset_index(drop=True)

df_group = df_group[['liability', 
                     'liability_label', 
                     'description',
                     'user_screen_name',
                     'retweet_count',
                     'favorite_count',
                     'count_followers', 
                     'tweet'
                     ]]

On retire les tweets parodiques.
Pour les autres tweets on remplace les valeurs liability et liability_label pour simplifier.

In [14]:
df_group = df_group.drop(df_group[df_group.liability == 1].index)
df_group.loc[df_group['liability'] == 2, 'liability_label'] = 'site plutôt douteux'
df_group.loc[df_group['liability'] == 3, 'liability_label'] = 'site plutôt douteux'
df_group.loc[df_group['liability'] == 4, 'liability_label'] = 'site plutôt fiable'
df_group.loc[df_group['liability'] == 2, 'liability'] = 1
df_group.loc[df_group['liability'] == 3, 'liability'] = 1
df_group.loc[df_group['liability'] == 4, 'liability'] = 0
df_group = df_group.reset_index(drop=True)

In [15]:
df_group

,liability,liability_label,description,user_screen_name,retweet_count,favorite_count,count_followers,tweet
0,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,50,101,98322,🚨Le professeur Raoult écope d’un blâme de l’Or...
1,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,643,0,98322,RT @DIVIZIO1: Recours déposé !
2,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,27,59,98322,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie..."
3,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,14,37,98322,"📚 Jonathan Curiel, directeur général adjoint d..."
4,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,39,79,98322,🖍 Le dessin du jour\n&gt; à retrouver sur http...
...,...,...,...,...,...,...,...,...
874677,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,0,2,73496,Do this and your cat will love you almost as m...
874678,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,0,1,73496,Tag a charity that you want your followers to ...
874679,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,1,3,73496,This article is for budding Virginia Woolfs AN...
874680,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,1,5,73496,Tried to make a pun about soccer in this tweet...


In [16]:
import copy
save_dataset = copy.deepcopy(df_group)

In [17]:
save_dataset

,liability,liability_label,description,user_screen_name,retweet_count,favorite_count,count_followers,tweet
0,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,50,101,98322,🚨Le professeur Raoult écope d’un blâme de l’Or...
1,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,643,0,98322,RT @DIVIZIO1: Recours déposé !
2,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,27,59,98322,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie..."
3,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,14,37,98322,"📚 Jonathan Curiel, directeur général adjoint d..."
4,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,39,79,98322,🖍 Le dessin du jour\n&gt; à retrouver sur http...
...,...,...,...,...,...,...,...,...
874677,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,0,2,73496,Do this and your cat will love you almost as m...
874678,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,0,1,73496,Tag a charity that you want your followers to ...
874679,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,1,3,73496,This article is for budding Virginia Woolfs AN...
874680,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,1,5,73496,Tried to make a pun about soccer in this tweet...


Supression des tweets anglais et italiens. 

In [18]:
! pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=af6a429d7f6dc5bedfb9e7956089afa75877c018a9bd3c34ea74d479b211a85a
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [19]:
! pip install advertools

     |████████████████████████████████| 287 kB 5.7 MB/s 
     |████████████████████████████████| 254 kB 40.7 MB/s 
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 251 kB 34.8 MB/s 
     |████████████████████████████████| 3.1 MB 32.9 MB/s 
     |████████████████████████████████| 3.6 MB 32.9 MB/s 
     |████████████████████████████████| 65 kB 2.7 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=7cfdfb109e571d1781e9286ecdffb990d05d1375b03938208593787efaf812be
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa69a339198e08bb63b7da2c45933a3428b29ec454
Successfully built PyDispatcher


In [20]:
#On récupère les stopwords italiens et anglais
from stop_words import get_stop_words
import advertools as adv
stop_words_en = get_stop_words('en')
stop_words_it = get_stop_words('it')
stop_words_arb = get_stop_words('arabic')
stop_words_jap = adv.stopwords['japanese']
stop_words_ch = adv.stopwords['chinese']



In [21]:
#On enlève les mots italiens trop commun avec des mots français pour éviter la confusion
stop_words_it.remove('tu')
stop_words_it.remove('un')
stop_words_it.remove('lui')
stop_words_it.remove('ma')
stop_words_it.remove('non')
stop_words_it.remove('si')
stop_words_it.remove('se')
stop_words_it.remove('qui')
stop_words_it.remove('te')
stop_words_it.remove('sui')
stop_words_it.remove('c')
stop_words_it.remove('e')
stop_words_it.remove('i')
stop_words_it.remove('l')
stop_words_it.remove('o')
stop_words_it.remove('è')
stop_words_it.remove('a')
stop_words_it.remove('le')
stop_words_it.remove('la')
stop_words_it.remove('il')

In [22]:
#Fonction pour détecter si la phrase est oui ou non italienne ou anglaise
def detect_language(tweet):
  tweet_toLow = tweet.lower()
  tweetList = tweet_toLow.split(" ")
  nbCommon_en = len(list(set(tweetList)&set(stop_words_en)))
  nbCommon_ar = len(list(set(tweetList)&set(stop_words_arb)))
  nbCommon_jp = len(list(set(tweetList)&set(stop_words_jap)))
  nbCommon_it = len(list(set(tweetList)&set(stop_words_it)))
  nbCommon_ch = len(list(set(tweetList)&set(stop_words_ch)))
  if nbCommon_en > 2 or nbCommon_it > 2 or nbCommon_ar > 2 or nbCommon_jp > 2 or nbCommon_ch > 2:
    return True
  else: 
    return False
  

In [23]:
#On supprime les éléments de la dataframe présentant des tweets anglais ou italiens
df_indexs= []
c = 0
for sentence in save_dataset['tweet']: 
  if detect_language(sentence):
      df_indexs.append(c)
  c = c+1
save_dataset = save_dataset.drop(labels=df_indexs, axis=0) 
save_dataset = save_dataset.reset_index(drop=True)

On sauvegarde le nettoyage des tweets dans un autre tableau

In [24]:
# df_tweets_clean = pd.DataFrame()
# df_tweets_clean['tweet'] = pd.DataFrame(df_group['tweet'])
df_tweets_clean = pd.DataFrame(save_dataset)

# reset index 
# df_tweets_clean.reset_index(drop=True)

# Mettre les mots en minuscule
# df_tweets_clean['minuscule'] = df_tweets_clean['tweet'].apply(lambda x : x.lower()) 
df_tweets_clean['cleaned'] = df_tweets_clean['tweet'].apply(lambda x : x.lower()) 

In [25]:
# transformation des liens internet en "desident_link"
# df_tweets_clean['liens'] = df_tweets_clean['nettoyer @'].apply(lambda x : re.sub(r"https://t.co/[a-zA-Z\u00c0-\u00FF0-9]+", "desident_link", x))
# df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"https://[a-zA-Z\u00c0-\u00FF0-9]+", "desident_link", x))

df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"http\S+", "desident_link", x))


In [26]:
# nettoyer hashtag #
# df_tweets_clean['nettoyer #'] = df_tweets_clean['minuscule'].apply(lambda x : re.sub(r"#\w+", "", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"#\S+", " ", x))

In [27]:
# nettoyer @
# df_tweets_clean['nettoyer @'] = df_tweets_clean['nettoyer #'].apply(lambda x : re.sub(r"@\w+", "", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"@\S+", " ", x))

In [28]:
# fonction nettoyer ponctuation
import string
def Remove_Punctuation(text):
    result = "".join([ch for ch in text if ch not in string.punctuation.replace('_','')])
    return result

In [29]:
# nettoyer les ponctuation
# df_tweets_clean['nettoyer ponctuaction'] = df_tweets_clean['nettoyer ?'].apply(lambda x: Remove_Punctuation(x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: Remove_Punctuation(x))

In [30]:
test = '@test toto'
re.sub(r"@\S+", " ", test)

'  toto'

In [31]:
# transformation des ! en "desident_exclamation"
# df_tweets_clean['nettoyer !'] = df_tweets_clean['liens'].apply(lambda x : re.sub(r"!", "desident_exclamation", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"!", "desident_exclamation", x))

In [32]:
# transformation des ? en "desident_exclamation"
# df_tweets_clean['nettoyer ?'] = df_tweets_clean['nettoyer !'].apply(lambda x : re.sub(r"[?]", "desident_question", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[?]", "desident_question", x))

Suppression des dates

In [33]:
patterns_date = [r'\blundi\b', r'\bmardi\b', r'\bmercredi\b', r'\bjeudi\b', r'\bvendredi\b', r'\bsamedi\b', r'\bdimanche\b',
                 r'\bjanvier\b', r'\bfévrier\b', r'\bfevrier\b', r'\bmars\b', r'\bavril\b', r'\bmai\b', r'\bjuin\b', r'\bjuillet\b', r'\baoût\b', r'\baout\b', r'\bseptembre\b', r'\boctobre\b', r'\bnovembre\b', r'\bdécembre\b', r'\bdecembre\b',
                 r'\bmatin\b', r'\bmidi\b', r'\bsoir\b', r'\bnuit\b', r'\b202[0-9]\b']
# print(patterns_date)

def Replace_date(tweet):
  for pattern in patterns_date:
    tweet = re.sub(pattern, ' desident_date ', tweet)
  return tweet

In [34]:
Replace_date("test demain, mai, 2023, test")

'test demain,  desident_date ,  desident_date , test'

In [35]:
df_tweets_clean["cleaned"] = df_tweets_clean["cleaned"].apply(lambda tweet : Replace_date(tweet))

In [36]:
df_tweets_clean.head(100)

,liability,liability_label,description,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
0,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,50,101,98322,🚨Le professeur Raoult écope d’un blâme de l’Or...,🚨le professeur raoult écope d’un blâme de l’or...
1,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,643,0,98322,RT @DIVIZIO1: Recours déposé !,rt recours déposé
2,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,27,59,98322,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie...",🎙best of pr michel drancourt ihu épidémies d...
3,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,14,37,98322,"📚 Jonathan Curiel, directeur général adjoint d...",📚 jonathan curiel directeur général adjoint de...
4,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,39,79,98322,🖍 Le dessin du jour\n&gt; à retrouver sur http...,🖍 le dessin du jour\ngt à retrouver sur deside...
...,...,...,...,...,...,...,...,...,...
95,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,43,92,98322,"Et notre entretien, à voir ou revoir 👇\nhttps:...",et notre entretien à voir ou revoir 👇\ndesiden...
96,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,100,193,98322,"📣 ""Nous voici donc entrés dans une nouvelle zo...",📣 nous voici donc entrés dans une nouvelle zon...
97,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,33,70,98322,⚠️ Dernière arnaque en date : l’UFC Que Choisi...,⚠️ dernière arnaque en date l’ufc que choisir...
98,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,114,256,98322,"🖍️ Le dessin du jour, par Z_caricature \n- à r...",🖍️ le dessin du jour par z_caricature \n à ret...


In [37]:
df_tweets_clean.loc[df_tweets_clean["cleaned"].str.contains("lenouveleco"), ["cleaned","tweet"]]

,cleaned,tweet


In [38]:
# transformation des valeurs numériques en "desident_number"
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"\b[0-9]+", " desident_number ", x))

In [39]:
# transformation des éléments de la forme d'une heure précise en "desident_date" (pour que les heures soient vu comme des indices de date)
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9]h[0-9][0-9]", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9][0-9]h[0-9][0-9]", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9][0-9]h", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"[0-9]h", "desident_date", x))

In [40]:
# transformation des éléments de la forme "desident_date" "desident_number" et "desident_number" "desident_date" en "desident_date"  (pour les formes comme "lundi 10" ou 25 décembre)
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"desident_date desident_number", "desident_date", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"desident_number desident_date", "desident_date", x))

In [41]:
# transformation des suites de "desident_date" en un seul "desident_date"
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub("(desident_date)+", "desident_date ", x))

In [42]:
df_tweets_clean.head(100)

,liability,liability_label,description,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
0,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,50,101,98322,🚨Le professeur Raoult écope d’un blâme de l’Or...,🚨le professeur raoult écope d’un blâme de l’or...
1,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,643,0,98322,RT @DIVIZIO1: Recours déposé !,rt recours déposé
2,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,27,59,98322,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie...",🎙best of pr michel drancourt ihu épidémies d...
3,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,14,37,98322,"📚 Jonathan Curiel, directeur général adjoint d...",📚 jonathan curiel directeur général adjoint de...
4,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,39,79,98322,🖍 Le dessin du jour\n&gt; à retrouver sur http...,🖍 le dessin du jour\ngt à retrouver sur deside...
...,...,...,...,...,...,...,...,...,...
95,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,43,92,98322,"Et notre entretien, à voir ou revoir 👇\nhttps:...",et notre entretien à voir ou revoir 👇\ndesiden...
96,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,100,193,98322,"📣 ""Nous voici donc entrés dans une nouvelle zo...",📣 nous voici donc entrés dans une nouvelle zon...
97,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,33,70,98322,⚠️ Dernière arnaque en date : l’UFC Que Choisi...,⚠️ dernière arnaque en date l’ufc que choisir...
98,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,114,256,98322,"🖍️ Le dessin du jour, par Z_caricature \n- à r...",🖍️ le dessin du jour par z_caricature \n à ret...


Suppression des émojis

In [43]:
# fonction (convert_emojis_to_text) : convertir les emojis en texte
# Entrée : text(un tweet)
# Sortie : text(après convertir)
def convert_emojis_to_text(text):
  # créer un objet : emot_obj 
  emot_obj = emot.core.emot()
  # les informations sont dans ce objet.emoji 
  emoji = emot_obj.emoji(text)
  # print('List emoji :     ', emoji) 
  
  # print('Text original :                 ', text)

  for i in range(len(emoji['value'])):
    text = re.sub(emoji['value'][i], " desident_emot_"+emoji['mean'][i]+" ", text)
    # Enlever les :
    text = re.sub(r':', '', text)

  # print('Text(convert emoji into text) : ', text)
  
  return text

In [ ]:
"""# nettoyer les emojis
# df_tweets_clean['emoji'] = df_tweets_clean['nettoyer ponctuaction'].apply(lambda x: convert_emojis_to_text(x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: convert_emojis_to_text(x))"""

"# nettoyer les emojis\n# df_tweets_clean['emoji'] = df_tweets_clean['nettoyer ponctuaction'].apply(lambda x: convert_emojis_to_text(x))\ndf_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: convert_emojis_to_text(x))"

In [44]:
spacy_stopwords_french = spacy.lang.fr.stop_words.STOP_WORDS  


# fonction Remove_Stopwords
def Remove_Stopwords(content_list):
  result = [word for word in content_list if word not in spacy_stopwords_french]
  return result

In [45]:
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : x.split())

In [46]:
# Suppression des stop words
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: Remove_Stopwords(x))

In [47]:
df_tweets_clean

,liability,liability_label,description,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
0,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,50,101,98322,🚨Le professeur Raoult écope d’un blâme de l’Or...,"[🚨le, professeur, raoult, écope, d’un, blâme, ..."
1,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,643,0,98322,RT @DIVIZIO1: Recours déposé !,"[rt, recours, déposé]"
2,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,27,59,98322,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie...","[🎙best, of, pr, michel, drancourt, ihu, épidém..."
3,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,14,37,98322,"📚 Jonathan Curiel, directeur général adjoint d...","[📚, jonathan, curiel, directeur, général, adjo..."
4,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,39,79,98322,🖍 Le dessin du jour\n&gt; à retrouver sur http...,"[🖍, dessin, jour, gt, retrouver, desident_link..."
...,...,...,...,...,...,...,...,...,...
722261,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,1,1,73496,@aubreyrumore wikiHow loves you more tho,"[wikihow, loves, you, more, tho]"
722262,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,8,19,73496,"""i.e. = """"id est"""" (that is)\ne.g. = """"exempli...","[ie, id, that, is, eg, exempli, gratia, for, t..."
722263,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,84,360,73496,@Thundercat https://t.co/5FYXI3t5Oa,[desident_link]
722264,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,0,2,73496,"""Throw some D's on that"" - ancient proverb htt...","[throw, some, ds, that, ancient, proverb, desi..."


In [48]:
# transmettre les liste de mots en phrase(texte)
def return_text(result):
  result_txt = " ".join(result)
  return result_txt

In [49]:
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x: return_text(x))

In [50]:
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"\b(gt)+", "", x))
df_tweets_clean['cleaned'] = df_tweets_clean['cleaned'].apply(lambda x : re.sub(r"\b[a-zA-Z\u00c0-\u00FF0-9]\b", "", x))

In [51]:
df_tweets_clean.sample(n=10, replace=False, random_state=seed)

,liability,liability_label,description,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned
95616,0,site plutôt fiable,Emission de télévision française présentée par...,Qofficiel,44,233,738647,🔇\n#Quotidien https://t.co/lt6B3zurCP,🔇 desident_link
435354,0,site plutôt fiable,Un hebdomadaire féminin.,Grazia,1,0,66227,Marni layered elegance #marni marni.official #...,marni layered elegance marniofficial desident_...
316824,0,site plutôt fiable,Un journal mensuel français d'information et d...,mdiplo,27,49,224069,2015. Deux économistes du FMI relèvent dans un...,desident_number économistes fmi relèvent rappo...
115485,0,site plutôt fiable,"Le site, québecois d'origine (la fondation de ...",PasseportSante,0,0,19964,7 idées de soupes pour prendre des forces avan...,desident_number idées soupes prendre forces lh...
534717,1,site plutôt douteux,Un site internet dédié à l'actualité médiatiqu...,morandiniblog,9,22,660218,DERNIERE MINUTE: La garde à vue du chanteur Je...,minute garde vue chanteur jeanluc lahaye soupç...
308511,0,site plutôt fiable,Un quotidien et site Internet d'information gé...,le_Parisien,3,5,2878495,Le régulateur européen approuve la vaccination...,régulateur européen approuve vaccination enfan...
654613,1,site plutôt douteux,Un site qui nous apparaît peu fiable et qui re...,WikistrikeW,0,0,6930,COVID-19 - Infectée par le coronavirus à la mi...,covid19 infectée coronavirus mimars ministre t...
337200,0,site plutôt fiable,Un quotidien et site Internet régional qui app...,ledauphine,0,2,87713,"Affaire Delphine Jubillar. Louis, 7 ans, a con...",affaire delphine jubillar louis desident_numbe...
686771,1,site plutôt douteux,Un site suisse proche de l'extrême droite épin...,ObservateursCH,2,1,5843,Europe « désouverainisée »: la Chine peut « no...,europe « désouverainisée » chine « mettre geno...
336102,0,site plutôt fiable,Un quotidien et site Internet d'information gé...,Le_Figaro,3,7,3465666,«L'écologie poétique de Julien Gracq apparaît ...,«lécologie poétique julien gracq apparaît mili...


In [54]:
doc = nlp("Je suis allé acheter du pain.")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
    


Je je PRON PRON__Number=Sing|Person=1 nsubj Xx True True
suis être AUX AUX__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin aux:pass xxxx True True
allé aller VERB VERB__VerbForm=Inf ROOT xxxx True False
acheter acheter VERB VERB__VerbForm=Inf xcomp xxxx True False
du de DET DET__Gender=Masc|Number=Sing det xx True True
pain pain NOUN NOUN__Gender=Masc|Number=Sing obj xxxx True False
. . PUNCT PUNCT___ punct . False False


spacy.tokens.doc.Doc

In [64]:
new_column = []
i=0
for msg in df_tweets_clean['cleaned']:
  if (i%1000==0):
    print('msg '+str(i))
  doc = nlp(msg)
  if (i%1000==0):
    print(doc)
  new_msg = ''
  for token in doc:
    new_msg = new_msg+token.text+'_'+token.pos_+' '
  if (i%1000==0):
    print(new_msg)
  new_column.append(new_msg)
  if (i%1000==0):
    print('\n\n')
  i = i+1

new_column

msg 0
🚨le professeur raoult écope ’un blâme ’ordre médecins article paraître desident_link
🚨_SCONJ le_DET professeur_NOUN raoult_PROPN écope_ADJ ’_PUNCT un_DET blâme_NOUN ’_PUNCT ordre_NOUN médecins_NOUN article_ADJ paraître_VERB desident_link_ADJ 



msg 1000
📣 soudain noyé épais brouillard gaz jen tombe genoux sol pleur… desident_link
📣_PROPN soudain_ADV noyé_VERB épais_ADJ brouillard_PROPN gaz_PROPN jen_ADJ tombe_VERB genoux_NOUN sol_NOUN pleur_NOUN …_PRON desident_link_NOUN 



msg 2000
monde foix desident_link
monde_NOUN foix_NOUN desident_link_ADJ 



msg 3000
📣 ’ivermectine déjà sauvé dizaines milliers vies gagne terrain ’industrie pharmaceutiqu… desident_link
📣_PROPN ’_PUNCT ivermectine_NOUN déjà_ADV sauvé_VERB dizaines_NOUN milliers_NOUN vies_NOUN gagne_VERB terrain_ADJ ’_PUNCT industrie_NOUN pharmaceutiqu_ADJ …_ADP desident_link_ADJ 



msg 4000
italien écoutez haut cardinal ’église ferai traduction en… desident_link
italien_ADJ écoutez_AUX haut_ADJ cardinal_NOUN ’_PUNCT égli

['🚨_SCONJ le_DET professeur_NOUN raoult_PROPN écope_ADJ ’_PUNCT un_DET blâme_NOUN ’_PUNCT ordre_NOUN médecins_NOUN article_ADJ paraître_VERB desident_link_ADJ ',
 'rt_DET recours_NOUN déposé_VERB ',
 '🎙_NOUN best_X of_X pr_X michel_PROPN drancourt_PROPN ihu_PROPN épidémies_NOUN dhier_PROPN daujourdhui_PROPN paléomicrobiologie_NOUN dac_NOUN …_PRON desident_link_NOUN ',
 '📚_PROPN jonathan_PROPN curiel_PROPN directeur_NOUN général_ADJ adjoint_NOUN programmes_NOUN chaînes_VERB m6_NOUN w9_PROPN desident_number_PROPN ter_X publié_VERB desident_date_ADV desident_date_ADV so_ADP …_ADV desident_link_NOUN ',
 '🖍_NOUN dessin_NOUN jour_NOUN  _SPACE retrouver_VERB desident_link_X desident_link_NOUN ',
 '🐖_PROPN l214_PROPN annoncé_VERB déposé_VERB plainte_NOUN mauvais_ADJ traitements_NOUN animaux_ADJ tromperie_NOUN consommateur_NOUN …_PRON desident_link_NOUN ',
 'soutien_NOUN dossier_NOUN audelà_ADJ porte_VERB plainte_NOUN  _SPACE réinfoliberté_PROPN qui_PRON …_ADV desident_link_NOUN ',
 '🎥_PROPN he

In [70]:
df_tweets_clean['tagged'] = new_column
df_tweets_clean

,liability,liability_label,description,user_screen_name,retweet_count,favorite_count,count_followers,tweet,cleaned,tagged
0,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,50,101,98322,🚨Le professeur Raoult écope d’un blâme de l’Or...,🚨le professeur raoult écope ’un blâme ’ordre m...,🚨_SCONJ le_DET professeur_NOUN raoult_PROPN éc...
1,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,643,0,98322,RT @DIVIZIO1: Recours déposé !,rt recours déposé,rt_DET recours_NOUN déposé_VERB
2,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,27,59,98322,"🎙BEST OF - Pr Michel Drancourt : IHU, épidémie...",🎙best of pr michel drancourt ihu épidémies dhi...,🎙_NOUN best_X of_X pr_X michel_PROPN drancourt...
3,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,14,37,98322,"📚 Jonathan Curiel, directeur général adjoint d...",📚 jonathan curiel directeur général adjoint pr...,📚_PROPN jonathan_PROPN curiel_PROPN directeur_...
4,1,site plutôt douteux,France-Soir était un journal fondé par Pierre ...,france_soir,39,79,98322,🖍 Le dessin du jour\n&gt; à retrouver sur http...,🖍 dessin jour retrouver desident_link desiden...,🖍_NOUN dessin_NOUN jour_NOUN _SPACE retrouver...
...,...,...,...,...,...,...,...,...,...,...
722261,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,1,1,73496,@aubreyrumore wikiHow loves you more tho,wikihow loves you more tho,wikihow_PROPN loves_X you_X more_X tho_ADJ
722262,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,8,19,73496,"""i.e. = """"id est"""" (that is)\ne.g. = """"exempli...",ie id that is eg exempli gratia for the sake o...,ie_X id_PROPN that_X is_X eg_X exempli_PROPN g...
722263,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,84,360,73496,@Thundercat https://t.co/5FYXI3t5Oa,desident_link,desident_link_NOUN
722264,0,site plutôt fiable,Un site qui donne des conseils et astuces sur ...,wikiHow,0,2,73496,"""Throw some D's on that"" - ancient proverb htt...",throw some ds that ancient proverb desident_li...,throw_X some_X ds_X that_X ancient_X proverb_X...


In [69]:
df_tweets_clean.to_csv('Tweets_nettoyés_150222.csv')
!cp Tweets_nettoyés_150222.csv "/content/drive/My Drive/PFE"

Pour l'enregistrement en format pickle

In [67]:
import pickle

In [68]:
df_tweets_clean.to_pickle(DATA_PATH+"Tweets_nettoyés_150222.pkl")